In [4]:
## 分析Cis 调控元件与优异变异间的联系，从趋势上来看，优异变异偏向于分布在cis元件附近
import pandas as pd
## 随机抽取假定cis元件位置，随机抽取假定的SNP位置，再计算二者overlap的数目
def sample_cis_snp_overlap(N):
    df = pd.DataFrame({'a':range(744678)})
    overlap_bin = []
    for i in range(N):
        df_cis = df.sample(145210)
        df_snp = df.sample(3139)
        df_overlap = pd.merge(df_cis, df_snp, how='inner', on=['a'])
        overlap_bin.append(df_overlap.shape[0])
    cis_snp_overlap = sum(overlap_bin)/N
    return cis_snp_overlap
sample_cis_snp_overlap(100)

612.41

In [21]:
## 将SNP转换为宽数据格式
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/Sup_Vari_Cis_header.bed", sep="\t")
for i in ['FE', 'FU', 'FL', 'FS']:
    output_f = "G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/{}_SNP_in_Cis.bed".format(i)
    df_tmp = df.query("tags==@i").copy()
    df_tmp.to_csv(output_f, sep="\t", index=False)
    df_tmp['SNP'] = df_tmp['chrom'].map(str) + "-" + df_tmp['SNP_start'].map(str) + "-" + df_tmp['SNP_end'].map(str)
    df_tmp.set_index("SNP", inplace=True)
    df_tmp_T = df_tmp.T
    df_tmp_T.drop(['chrom', 'SNP_start', 'SNP_end', 'tags'], axis=0, inplace=True)
    df_tmp_T.to_csv(output_f, sep="\t")

In [39]:
## 分析有利变异，1、统计，有多少栽培种和农家种发生了有利变异
import pandas as pd
def statistics_benefit_mutation():
    df_name = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/tree_group.txt", sep="\t", names=['sample', 'tags'])
    for i in ['FE', 'FU', 'FL', 'FS']:
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/{}_SNP_in_Cis.bed".format(i)
        output_f = "G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/{}_SNP_in_Cis_statistics.bed".format(i)
        df = pd.read_csv(input_f, sep="\t")
        ## 将宽数据转换为长数据
        df_long = pd.melt(df, id_vars='sample')
        ## 将所有的2替换为1， -1替换为0
        old_tags = [2, -1]
        new_tags = [1, 0]
        df_long['value'].replace(old_tags, new_tags, inplace=True)
        ## 给材料加上 农家种和栽培种的标签
        df_long.columns = ['sample', 'snp', 'value']
        df_long = pd.merge(df_long, df_name, how='inner', on=['sample'])
        df_statistics = df_long.groupby(by=['tags', 'snp'], as_index=False)['value'].sum()
        df_cultivar = df_statistics.query("tags=='cultivar'").copy()
        df_landrace = df_statistics.query("tags=='landrace'").copy()
        df_cultivar['ratio'] = df_cultivar['value']/3220
        df_landrace['ratio'] = df_landrace['value']/332
        df_return = pd.concat([df_cultivar, df_landrace], axis=0)
        df_return.to_csv(output_f, sep="\t", index=False)
statistics_benefit_mutation()

In [28]:
## 统计有利变异的数目
import pandas as pd
import numpy as np
def SNP_number(df):
    df = df.copy()
    df_wide = df.pivot(columns='tags', index='snp', values='ratio')
    df_wide['ratio'] = np.abs(np.log2(df_wide['cultivar']/df_wide['landrace']))
    df_return = df_wide.query('ratio>1').copy()
    return df_return
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/FL_SNP_in_Cis_statistics.bed", sep="\t")
df_return = SNP_number(df)
df_active = df_return.query("cultivar>landrace")
print(df_active.shape[0])

52


In [46]:
## 统计有利变异的数目
import pandas as pd
import numpy as np
def SNP_number(df):
    df = df.copy()
    df_wide = df.pivot(columns='tags', index='snp', values='ratio')
    df_wide['ratio'] = np.abs(np.log2((df_wide['cultivar'])/(df_wide['landrace'])))
    df_return = df_wide.query('ratio>1').copy()
    print(df_return.shape[0])
    df_active = df_return.query("cultivar>landrace").copy()
    df_deactive = df_return.query("cultivar<landrace").copy()
    print(df_active.shape[0], df_deactive.shape[0])
    return df_return

for i in ['FE', 'FU', 'FL', 'FS']:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/{}_SNP_in_Cis_statistics.bed".format(i)
    output_f = "G:/Billfish/J668_multip_tissue_3D-genome/mutation/QZY/superior_varition/{}_SNP_in_Cis_choosed.bed".format(i)
    df = pd.read_csv(input_f, sep="\t")
    df_return = SNP_number(df)
    df_return.to_csv(output_f, sep="\t")

256
161 95
89
52 37
97
52 45
213
106 107
